In [6]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts


from mne_connectivity import spectral_connectivity_epochs
from mne.datasets import sample
import seaborn as sns

from scipy.signal import welch
import yasa
import constants
import numpy as np

In [5]:
power_bands = {'delta' : (0.5,4) , 'theta' : (4,8) , 'alpha' : (8,12) , 'sigma' : (12,16) , 'beta' : (16,30) , 'gamma' : (30,40) }
power_band_names = [val for val in power_bands.keys()]
folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/Connectivity/'
X = pd.read_hdf(folder + 'Wake_coherence_df.h5')
y = pd.read_hdf('/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/C_22_data/Full_dfs/' + 'Wake_y.h5')
groups = pd.read_hdf('/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/C_22_data/Full_dfs/' + 'Wake_groups.h5')

In [49]:
X[[col for col in X.columns if 'delta' in col]]

delta_Fp1_Fpz  delta_Fp1_Fp2  delta_Fp1_F1  delta_Fp1_Fz  delta_Fp1_F2  \
0       0.953931       0.520429      0.542333      0.348677      0.353381   
0       0.635187       0.621240      0.282571      0.185618      0.304530   
0       0.938134       0.870624      0.598293      0.338438      0.719192   
0       0.869746       0.716190      0.372368      0.341759      0.566200   
0       0.902964       0.597608      0.300778      0.323164      0.411252   
0       0.771525       0.573305      0.482682      0.350296      0.326987   
0       0.914142       0.719624      0.409281      0.282655      0.419215   
0       0.852841       0.412237      0.310094      0.375978      0.431138   
0       0.715879       0.600220      0.244397      0.139771      0.397500   
0       0.757792       0.594291      0.453150      0.338333      0.408027   
0       0.626455       0.492965      0.223820      0.308070      0.484146   
0       0.936050       0.588320      0.506206      0.352826      0.511443   
0       0.946460       0.555457      0.412652      0.399265      0.539441   
0       0.830808       0.337279      0.258378      0.521633      0.450608   
0       0.592915       0.478837      0.137822      0.134504      0.319969   
0       0.828920       0.688291      0.503233      0.209490      0.280818   
0       0.858241       0.678323      0.287139      0.273313      0.585986   
0       0.605814       0.637227      0.196953      0.127044      0.292101   
0       0.734355       0.690379      0.278865      0.175774      0.547098   
0       0.753693       0.502889      0.234738      0.183405      0.374926   
0       0.719185       0.609116      0.257905      0.269007      0.519175   
0       0.617066       0.545765      0.184397      0.127459      0.343721   
0       0.535765       0.437829      0.196788      0.138430      0.302973   
0       0.661084       0.666672      0.383720      0.185034      0.325042   
0       0.528141       0.745333      0.136244      0.194115      0.470704   
0       0.891359       0.458615      0.272874      0.332242      0.557080   
0       0.936439       0.795294      0.472311      0.406793      0.380206   
0       0.638484       0.741053      0.218733      0.098662      0.386140   
0       0.823804       0.648331      0.407677      0.130815      0.300161   
0       0.908922       0.785836      0.647703      0.509056      0.711931   
0       0.918556       0.695576      0.678881      0.471204      0.652577   
0       0.876928       0.769733      0.538107      0.386600      0.695626   
0       0.756993       0.599131      0.371709      0.168881      0.333271   
0       0.950644       0.589514      0.515216      0.275087      0.459219   
0       0.934350       0.930468      0.588173      0.271638      0.674159   
0       0.978513       0.890836      0.756138      0.599226      0.772464   
0       0.854134       0.639100      0.297817      0.207073      0.449913   
0       0.839549       0.721900      0.274833      0.301961      0.548685   
0       0.606438       0.585602      0.206879      0.123498      0.371660   
0       0.867812       0.753898      0.552785      0.155337      0.228138   
0       0.899192       0.638651      0.390413      0.284325      0.400997   
0       0.607428       0.535881      0.392302      0.261136      0.325770   
0       0.445143       0.424270      0.281522      0.183944      0.324887   
0       0.970608       0.750486      0.575673      0.444377      0.473054   
0       0.273060       0.586358      0.162789      0.137807      0.414126   
0       0.877093       0.642887      0.480258      0.216598      0.294791   
0       0.687406       0.531621      0.205062      0.123443      0.435511   
0       0.956078       0.666006      0.544234      0.348933      0.521651   
0       0.825914       0.664830      0.359264      0.337727      0.465100   
0       0.616399       0.655091      0.299516      0.156777      0.323812   
0       0.702968       0.702592      0.447197      0.139012      0.237554   

In [52]:
t1 = time.time()
for power_band in list(power_bands.keys()) +['_']:
    print(power_band)
    acc_scores = []
    X_subset = X[[col for col in X.columns if power_band in col]]
    for i in range(10):
        clf = RandomForestClassifier()
        X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.2)
        
        clf.fit(X_train, y_train) 
        y_pred = clf.predict(X_test)
        acc_scores.append(str(accuracy_score(y_test, y_pred ))[:4] )
    
    print(acc_scores)    
    
t2 = time.time()

t2-t1

delta
['0.41', '0.41', '0.33', '0.16', '0.33', '0.25', '0.5', '0.33', '0.41', '0.33']
theta
['0.33', '0.41', '0.08', '0.41', '0.33', '0.33', '0.33', '0.33', '0.08', '0.41']
alpha
['0.58', '0.25', '0.5', '0.58', '0.41', '0.25', '0.41', '0.41', '0.16', '0.41']
sigma
['0.5', '0.33', '0.33', '0.33', '0.41', '0.58', '0.41', '0.33', '0.25', '0.33']
beta
['0.5', '0.33', '0.16', '0.41', '0.16', '0.33', '0.33', '0.41', '0.5', '0.25']
gamma
['0.58', '0.41', '0.33', '0.5', '0.33', '0.41', '0.33', '0.33', '0.33', '0.5']
_
['0.25', '0.16', '0.33', '0.66', '0.5', '0.41', '0.08', '0.5', '0.41', '0.5']


10.252922773361206

## Expt 1 

In [63]:
X = X.reset_index(drop = True)

In [79]:
t1 = time.time()
#1.Generate expt specific X,y,groups
X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 1)

for power_band in list(power_bands.keys()) +['_']:
    print(power_band)
    acc_scores = []
    X_subset = X_expt[[col for col in X_expt.columns if power_band in col]]
    for i in range(10):
        clf = RandomForestClassifier()
        X_train, X_test, y_train, y_test = train_test_split(X_subset, y_expt, test_size=0.2)
        
        clf.fit(X_train, y_train) 
        y_pred = clf.predict(X_test)
        acc_scores.append(str(accuracy_score(y_test, y_pred ))[:4] )
    
    print(np.array([float(acc) for acc in acc_scores]).mean())
    print(acc_scores)    
    
t2 = time.time()

t2-t1

delta
0.49700000000000005
['0.33', '0.33', '0.33', '0.33', '0.33', '1.0', '0.5', '0.33', '0.83', '0.66']
theta
0.5650000000000001
['0.5', '0.5', '0.66', '0.83', '0.83', '0.5', '0.5', '0.5', '0.5', '0.33']
alpha
0.562
['0.33', '0.83', '0.33', '0.66', '0.66', '0.66', '0.66', '0.33', '0.66', '0.5']
sigma
0.48100000000000004
['0.5', '0.33', '0.5', '0.5', '0.16', '0.83', '0.66', '0.5', '0.5', '0.33']
beta
0.6960000000000001
['0.66', '0.83', '0.66', '0.83', '0.5', '0.83', '0.33', '0.83', '0.83', '0.66']
gamma
0.6620000000000001
['0.66', '0.66', '0.33', '0.66', '0.83', '0.83', '0.5', '0.66', '0.66', '0.83']
_
0.6130000000000001
['0.66', '0.66', '0.83', '0.5', '0.5', '0.66', '0.5', '0.66', '0.5', '0.66']


8.71087908744812

# Expt 2

In [78]:
t1 = time.time()
#1.Generate expt specific X,y,groups
X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 2 )

for power_band in list(power_bands.keys()) +['_']:
    print(power_band)
    acc_scores = []
    X_subset = X_expt[[col for col in X_expt.columns if power_band in col]]
    for i in range(10):
        clf = RandomForestClassifier()
        X_train, X_test, y_train, y_test = train_test_split(X_subset, y_expt, test_size=0.2)
        
        clf.fit(X_train, y_train) 
        y_pred = clf.predict(X_test)
        acc_scores.append(str(accuracy_score(y_test, y_pred ))[:4] )
        # print(np.mean(np.array(acc_scores)))
    
    print(acc_scores)    
    print(np.array([float(acc) for acc in acc_scores]).mean())
t2 = time.time()

t2-t1

delta
['0.62', '0.62', '0.75', '0.62', '0.75', '0.5', '0.37', '0.5', '0.62', '0.75']
0.6100000000000001
theta
['0.5', '0.62', '0.5', '0.37', '0.5', '0.5', '0.5', '0.25', '0.62', '0.75']
0.511
alpha
['0.37', '0.37', '0.12', '0.37', '0.5', '0.5', '0.37', '0.62', '0.25', '0.37']
0.384
sigma
['0.25', '0.62', '0.5', '0.0', '0.62', '0.62', '0.5', '0.75', '0.37', '0.5']
0.47300000000000003
beta
['0.12', '0.5', '0.25', '0.37', '0.37', '0.25', '0.37', '0.5', '0.25', '0.25']
0.323
gamma
['0.62', '0.37', '0.37', '0.5', '0.37', '0.25', '0.62', '0.5', '0.62', '0.75']
0.497
_
['0.62', '0.62', '0.62', '0.37', '0.75', '0.62', '0.62', '0.75', '0.25', '0.37']
0.559


9.002055644989014

# Expt 3

In [80]:
t1 = time.time()
#1.Generate expt specific X,y,groups
X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 3)

for power_band in list(power_bands.keys()) +['_']:
    print(power_band)
    acc_scores = []
    X_subset = X_expt[[col for col in X_expt.columns if power_band in col]]
    for i in range(10):
        clf = RandomForestClassifier()
        X_train, X_test, y_train, y_test = train_test_split(X_subset, y_expt, test_size=0.2)
        
        clf.fit(X_train, y_train) 
        y_pred = clf.predict(X_test)
        acc_scores.append(str(accuracy_score(y_test, y_pred ))[:4] )
    
    print(acc_scores)    
    print(np.array([float(acc) for acc in acc_scores]).mean())

t2 = time.time()

t2-t1

delta
['0.5', '0.41', '0.41', '0.41', '0.41', '0.41', '0.41', '0.33', '0.5', '0.41']
0.42000000000000004
theta
['0.41', '0.58', '0.58', '0.41', '0.5', '0.33', '0.41', '0.41', '0.5', '0.5']
0.46299999999999997
alpha
['0.16', '0.33', '0.16', '0.16', '0.25', '0.33', '0.08', '0.25', '0.25', '0.33']
0.23000000000000004
sigma
['0.33', '0.25', '0.41', '0.41', '0.25', '0.5', '0.5', '0.41', '0.25', '0.16']
0.347
beta
['0.5', '0.41', '0.41', '0.25', '0.33', '0.33', '0.16', '0.58', '0.25', '0.33']
0.355
gamma
['0.25', '0.5', '0.41', '0.33', '0.33', '0.16', '0.0', '0.33', '0.25', '0.41']
0.29700000000000004
_
['0.33', '0.5', '0.33', '0.41', '0.33', '0.25', '0.33', '0.5', '0.5', '0.33']
0.38100000000000006


10.166969776153564

# Expt 4

In [81]:
t1 = time.time()
#1.Generate expt specific X,y,groups
X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 4)

for power_band in list(power_bands.keys()) +['_']:
    print(power_band)
    acc_scores = []
    X_subset = X_expt[[col for col in X_expt.columns if power_band in col]]
    for i in range(10):
        clf = RandomForestClassifier()
        X_train, X_test, y_train, y_test = train_test_split(X_subset, y_expt, test_size=0.2)
        
        clf.fit(X_train, y_train) 
        y_pred = clf.predict(X_test)
        acc_scores.append(str(accuracy_score(y_test, y_pred ))[:4] )
    
    print(acc_scores)    
    print(np.array([float(acc) for acc in acc_scores]).mean())

t2 = time.time()

t2-t1

delta
['0.33', '0.16', '0.41', '0.16', '0.41', '0.41', '0.58', '0.33', '0.5', '0.41']
0.37
theta
['0.16', '0.16', '0.16', '0.33', '0.16', '0.25', '0.41', '0.33', '0.16', '0.16']
0.22800000000000004
alpha
['0.33', '0.58', '0.08', '0.16', '0.33', '0.33', '0.16', '0.25', '0.25', '0.16']
0.263
sigma
['0.5', '0.25', '0.25', '0.16', '0.5', '0.5', '0.41', '0.41', '0.16', '0.33']
0.34700000000000003
beta
['0.33', '0.33', '0.25', '0.25', '0.5', '0.33', '0.25', '0.5', '0.5', '0.16']
0.34
gamma
['0.41', '0.16', '0.25', '0.33', '0.5', '0.41', '0.58', '0.41', '0.41', '0.41']
0.387
_
['0.33', '0.25', '0.41', '0.16', '0.5', '0.58', '0.41', '0.58', '0.25', '0.41']
0.388


10.26976490020752